# Linear regression

This notebook compares various linear regression implementations. The dataset used is the [Toulouse bike sharing dataset](https://online-ml.github.io/generated/river.datasets.fetch_bikes.html#river.datasets.fetch_bikes).

In [1]:
%load_ext watermark
%watermark --python --machine --packages river,sklearn,torch,vowpalwabbit --datename

Tue Nov 24 2020 

CPython 3.8.5
IPython 7.19.0

river 0.1.0
sklearn 0.23.2
torch 1.7.0
vowpalwabbit unknown

compiler   : Clang 10.0.0 
system     : Darwin
release    : 20.1.0
machine    : x86_64
processor  : i386
CPU cores  : 8
interpreter: 64bit


Common parameters.

In [2]:
N_FEATURES = 6
LEARNING_RATE = 0.005

River model.

In [3]:
from river import linear_model
from river import optim

river_model = linear_model.LinearRegression(
    optimizer=optim.SGD(LEARNING_RATE),
    l2=0.,
    intercept_lr=LEARNING_RATE
)

scikit-learn model.

In [4]:
from river import compat
from sklearn import linear_model

sklearn_model = compat.SKL2RiverRegressor(
    linear_model.SGDRegressor(
        learning_rate='constant',
        eta0=LEARNING_RATE,
        penalty='none'
    )
)

PyTorch model.

In [5]:
import torch

class PyTorchNet(torch.nn.Module):
    
    def __init__(self, n_features):
        super().__init__()
        self.linear = torch.nn.Linear(N_FEATURES, 1)
        torch.nn.init.constant_(self.linear.weight, 0)
        torch.nn.init.constant_(self.linear.bias, 0)
        
    def forward(self, x):
        return self.linear(x)
    
torch_model = PyTorchNet(n_features=N_FEATURES)
torch_model = compat.PyTorch2RiverRegressor(
    net=torch_model,
    loss_fn=torch.nn.MSELoss(),
    optimizer=torch.optim.SGD(torch_model.parameters(), lr=LEARNING_RATE)
)

In [6]:
from river import compose
from river import datasets
from river import feature_extraction
from river import metrics
from river import preprocessing
from river import stats

%run utils.py

def add_hour(x):
    x['hour'] = x['moment'].hour
    return x

results = benchmark(
    get_X_y=datasets.Bikes,
    n=182_470,
    get_pp=lambda: (
        compose.Select('clouds', 'humidity', 'pressure', 'temperature', 'wind') +
        (
            add_hour |
            feature_extraction.TargetAgg(by=['station', 'hour'], how=stats.Mean())
        ) |
        preprocessing.StandardScaler()
    ),
    models=[
        ('river', 'LinearRegression', river_model),
        ('scikit-learn', 'SGDRegressor', sklearn_model),
        ('PyTorch (CPU)', 'Linear', torch_model),
    ],
    get_metric=metrics.MSE
)

100%|██████████| 3/3 [02:48<00:00, 56.33s/it]


In [7]:
results

,Library,Model,MSE,Fit time,Average fit time,Predict time,Average predict time
0,river,LinearRegression,23.037118,"3s, 588ms, 133μs, 755ns","19μs, 664ns","465ms, 173μs, 863ns","2μs, 549ns"
1,scikit-learn,SGDRegressor,25.296774,"33s, 614ms, 256μs, 445ns","184μs, 218ns","14s, 985ms, 462μs, 445ns","82μs, 126ns"
2,PyTorch (CPU),Linear,23.037118,"45s, 505ms, 615μs, 146ns","249μs, 387ns","13s, 422ms, 835μs, 437ns","73μs, 562ns"
